# Combing the Tuned Lens and the Transformer Lens
asdfdsafasdfsadfdsafasdffds

In [1]:
import torch as th
from tuned_lens.plotting import PredictionTrajectory
from tuned_lens.nn import TunedLens, Unembed
import transformer_lens as tl

model = tl.HookedTransformer.from_pretrained(
    "EleutherAI/pythia-160m-deduped-v0",
    device="cpu"
)
assert model.tokenizer is not None


lens = TunedLens.from_unembed_and_pretrained(
    unembed=Unembed(model),
    lens_resource_id="EleutherAI/pythia-160m-deduped-v0",
)

def to_targets(input_ids: th.Tensor):
    print(f"{input_ids[..., 1:].shape=} {th.full(input_ids.shape[:-1] + (1,), model.tokenizer.eos_token_id).shape=}")
    return th.cat(
        (input_ids[..., 1:], th.full(input_ids.shape[:-1] + (1,), model.tokenizer.eos_token_id)
    ), dim=-1)


Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-160m-deduped-v0 into HookedTransformer


In [2]:
assert model.tokenizer
input_ids = model.to_tokens(["Alex and John entered the room. John turned to Alex", "Bob and Alice entered the room. Alice turned to Bob"])
targets = to_targets(input_ids)
with th.inference_mode():
    logits, cache = model.run_with_cache(
        input=input_ids, return_type="logits"
    )

    pred_traj_clean = PredictionTrajectory.from_lens_and_cache(
        lens=lens,
        cache=cache,
        model_logits=logits,
        input_ids=input_ids,
        targets=targets,
    )

input_ids[..., 1:].shape=torch.Size([2, 11]) th.full(input_ids.shape[:-1] + (1,), model.tokenizer.eos_token_id).shape=torch.Size([2, 1])


In [3]:
pred_traj_clean.cross_entropy().figure()

['<|endoftext|>', '*', 'Ġand', '*', 'Ġentered', 'Ġthe', 'Ġroom', '.', '*', 'Ġturned', 'Ġto', '*']
